## 1. Importación de Librerías

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2. Configuración de Rutas

In [ ]:
ruta_base = Path(r'e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\16S03. Proyecto 03\P3-EcoSort')
ruta_features = ruta_base / 'result' / 'features'
ruta_modelos = ruta_base / 'result' / 'models'
ruta_figuras = ruta_base / 'result' / 'figures'

clases = ['general', 'paper', 'plastic']

## 3. Carga de Datos y Modelos

### 3.1 Carga de Características

In [ ]:
X_val_img = np.load(ruta_features / 'X_val_imagenes.npy')
y_val = np.load(ruta_features / 'y_val.npy')
X_val_pca = np.load(ruta_features / 'features_val_pca.npy')

df_datos = pd.DataFrame({
    'Dataset': ['Validación Imágenes', 'Validación PCA'],
    'Shape': [X_val_img.shape, X_val_pca.shape],
    'Etiquetas': [len(y_val), len(y_val)]
})

df_datos

### 3.2 Carga de Modelos y Métricas

In [ ]:
modelo_lr = joblib.load(ruta_modelos / 'logistic_regression.pkl')
metricas_lr = joblib.load(ruta_modelos / 'logistic_regression_metricas.pkl')

modelo_svm = joblib.load(ruta_modelos / 'svm.pkl')
metricas_svm = joblib.load(ruta_modelos / 'svm_metricas.pkl')

historial_cnn = joblib.load(ruta_modelos / 'cnn_historial.pkl')
metricas_cnn = joblib.load(ruta_modelos / 'cnn_metricas.pkl')

### 3.3 Reconstrucción del Modelo CNN

In [ ]:
class CNN_Clasificador(nn.Module):
    def __init__(self, num_clases=3, dropout=0.5):
        super(CNN_Clasificador, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 16 * 16, 512)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(128, num_clases)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

modelo_cnn = CNN_Clasificador(num_clases=3, dropout=0.5).to(device)
modelo_cnn.load_state_dict(torch.load(ruta_modelos / 'cnn_model.pth'))
modelo_cnn.eval()

## 4. Comparación de Métricas

In [ ]:
df_comparacion = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'SVM', 'CNN'],
    'F1 Macro Train': [
        metricas_lr['f1_macro_train'],
        metricas_svm['f1_macro_train'],
        metricas_cnn['f1_macro_train']
    ],
    'F1 Macro Val': [
        metricas_lr['f1_macro_val'],
        metricas_svm['f1_macro_val'],
        metricas_cnn['f1_macro_val']
    ],
    'F1 Micro Val': [
        metricas_lr['f1_micro_val'],
        metricas_svm['f1_micro_val'],
        metricas_cnn['f1_micro_val']
    ],
    'AUC-PR Val': [
        metricas_lr['auc_pr_macro_val'],
        metricas_svm['auc_pr_macro_val'],
        metricas_cnn['auc_pr_macro_val']
    ]
})

df_comparacion['Overfitting'] = (df_comparacion['F1 Macro Train'] - df_comparacion['F1 Macro Val']).abs()
df_comparacion = df_comparacion.round(4)

df_comparacion

### 4.1 Visualización Comparativa

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

modelos = df_comparacion['Modelo']
colores = ['#3498db', '#2ecc71', '#e74c3c']

axes[0, 0].bar(modelos, df_comparacion['F1 Macro Val'], alpha=0.8, color=colores, edgecolor='black')
axes[0, 0].set_ylabel('F1 Macro Score', fontsize=12, fontweight='bold')
axes[0, 0].set_title('F1 Macro - Validación', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate(df_comparacion['F1 Macro Val']):
    axes[0, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

axes[0, 1].bar(modelos, df_comparacion['F1 Micro Val'], alpha=0.8, color=colores, edgecolor='black')
axes[0, 1].set_ylabel('F1 Micro Score', fontsize=12, fontweight='bold')
axes[0, 1].set_title('F1 Micro - Validación', fontsize=14, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].set_ylim([0, 1])
for i, v in enumerate(df_comparacion['F1 Micro Val']):
    axes[0, 1].text(i, v + 0.02, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

axes[1, 0].bar(modelos, df_comparacion['AUC-PR Val'], alpha=0.8, color=colores, edgecolor='black')
axes[1, 0].set_ylabel('AUC-PR Score', fontsize=12, fontweight='bold')
axes[1, 0].set_title('AUC-PR Macro - Validación', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].set_ylim([0, 1])
for i, v in enumerate(df_comparacion['AUC-PR Val']):
    axes[1, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

axes[1, 1].bar(modelos, df_comparacion['Overfitting'], alpha=0.8, color=colores, edgecolor='black')
axes[1, 1].set_ylabel('Diferencia F1 (Train - Val)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Indicador de Overfitting', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(df_comparacion['Overfitting']):
    axes[1, 1].text(i, v + 0.005, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '04_eval_01_comparacion_metricas.svg', format='svg', bbox_inches='tight')
plt.show()

## 5. Matrices de Confusión Comparativas

In [ ]:
y_pred_lr = modelo_lr.predict(X_val_pca)
y_pred_svm = modelo_svm.predict(X_val_pca)

X_val_tensor = torch.FloatTensor(X_val_img).permute(0, 3, 1, 2)
y_val_tensor = torch.LongTensor(y_val)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

y_pred_cnn = []
with torch.no_grad():
    for inputs, _ in val_loader:
        inputs = inputs.to(device)
        outputs = modelo_cnn(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred_cnn.extend(predicted.cpu().numpy())

y_pred_cnn = np.array(y_pred_cnn)

cm_lr = confusion_matrix(y_val, y_pred_lr)
cm_svm = confusion_matrix(y_val, y_pred_svm)
cm_cnn = confusion_matrix(y_val, y_pred_cnn)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', xticklabels=clases, yticklabels=clases,
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=axes[0])
axes[0].set_xlabel('Predicción', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Valor Real', fontsize=11, fontweight='bold')
axes[0].set_title(f'Logistic Regression\nF1 Macro: {metricas_lr["f1_macro_val"]:.4f}', 
                 fontsize=12, fontweight='bold')

sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', xticklabels=clases, yticklabels=clases,
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=axes[1])
axes[1].set_xlabel('Predicción', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Valor Real', fontsize=11, fontweight='bold')
axes[1].set_title(f'SVM\nF1 Macro: {metricas_svm["f1_macro_val"]:.4f}', 
                 fontsize=12, fontweight='bold')

sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Reds', xticklabels=clases, yticklabels=clases,
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=axes[2])
axes[2].set_xlabel('Predicción', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Valor Real', fontsize=11, fontweight='bold')
axes[2].set_title(f'CNN\nF1 Macro: {metricas_cnn["f1_macro_val"]:.4f}', 
                 fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '04_eval_02_matrices_confusion.svg', format='svg', bbox_inches='tight')
plt.show()

## 6. Curvas de Pérdida del Modelo CNN

In [ ]:
train_loss_cnn = historial_cnn['train_loss']
val_loss_cnn = historial_cnn['val_loss']
train_acc_cnn = historial_cnn['train_acc']
val_acc_cnn = historial_cnn['val_acc']

epochs = range(1, len(train_loss_cnn) + 1)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(epochs, train_loss_cnn, 'b-', linewidth=2, label='Train Loss', marker='o', markersize=3)
axes[0].plot(epochs, val_loss_cnn, 'r-', linewidth=2, label='Validation Loss', marker='s', markersize=3)
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0].set_title('Curvas de Pérdida - CNN', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11, loc='best')
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs, train_acc_cnn, 'b-', linewidth=2, label='Train Accuracy', marker='o', markersize=3)
axes[1].plot(epochs, val_acc_cnn, 'r-', linewidth=2, label='Validation Accuracy', marker='s', markersize=3)
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].set_title('Curvas de Accuracy - CNN', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11, loc='best')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ruta_figuras / '04_eval_03_curvas_cnn.svg', format='svg', bbox_inches='tight')
plt.show()

### 6.1 Análisis de Convergencia

In [ ]:
mejor_epoch = np.argmin(val_loss_cnn) + 1
mejor_val_loss = val_loss_cnn[mejor_epoch - 1]
mejor_val_acc = val_acc_cnn[mejor_epoch - 1]

diferencia_final = train_loss_cnn[-1] - val_loss_cnn[-1]

df_convergencia = pd.DataFrame({
    'Métrica': [
        'Mejor Epoch',
        'Mejor Val Loss',
        'Mejor Val Accuracy',
        'Loss Final Train',
        'Loss Final Val',
        'Diferencia Final (Train-Val)',
        'Estado'
    ],
    'Valor': [
        mejor_epoch,
        f'{mejor_val_loss:.4f}',
        f'{mejor_val_acc:.4f}',
        f'{train_loss_cnn[-1]:.4f}',
        f'{val_loss_cnn[-1]:.4f}',
        f'{diferencia_final:.4f}',
        'Overfitting leve' if diferencia_final > 0.1 else 'Buen ajuste'
    ]
})

df_convergencia

## 7. Ranking de Modelos

In [ ]:
df_ranking = df_comparacion.copy()

df_ranking['Rank F1 Macro'] = df_ranking['F1 Macro Val'].rank(ascending=False)
df_ranking['Rank F1 Micro'] = df_ranking['F1 Micro Val'].rank(ascending=False)
df_ranking['Rank AUC-PR'] = df_ranking['AUC-PR Val'].rank(ascending=False)
df_ranking['Rank Overfitting'] = df_ranking['Overfitting'].rank(ascending=True)

pesos = {'F1 Macro': 0.5, 'F1 Micro': 0.2, 'AUC-PR': 0.2, 'Overfitting': 0.1}

df_ranking['Score Total'] = (
    df_ranking['Rank F1 Macro'] * pesos['F1 Macro'] +
    df_ranking['Rank F1 Micro'] * pesos['F1 Micro'] +
    df_ranking['Rank AUC-PR'] * pesos['AUC-PR'] +
    df_ranking['Rank Overfitting'] * pesos['Overfitting']
)

df_ranking['Ranking Final'] = df_ranking['Score Total'].rank(ascending=False).astype(int)
df_ranking = df_ranking.sort_values('Ranking Final')

df_ranking_display = df_ranking[[
    'Ranking Final', 'Modelo', 'F1 Macro Val', 'F1 Micro Val', 'AUC-PR Val', 'Overfitting'
]]

df_ranking_display

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

modelos_ordenados = df_ranking['Modelo'].values
scores = df_ranking['Score Total'].values

colores_ranking = ['#2ecc71', '#f39c12', '#e74c3c']
bars = ax.barh(modelos_ordenados, scores, alpha=0.8, color=colores_ranking, edgecolor='black')

ax.set_xlabel('Score Total Ponderado', fontsize=12, fontweight='bold')
ax.set_title('Ranking Final de Modelos (Menor es Mejor)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

for i, (modelo, score, rank) in enumerate(zip(modelos_ordenados, scores, df_ranking['Ranking Final'])):
    ax.text(score + 0.02, i, f'#{rank} - Score: {score:.3f}', 
           va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '04_eval_04_ranking_modelos.svg', format='svg', bbox_inches='tight')
plt.show()

## 8. Reportes de Clasificación Detallados

In [ ]:
from sklearn.metrics import classification_report

reporte_lr = classification_report(y_val, y_pred_lr, target_names=clases, output_dict=True)
reporte_svm = classification_report(y_val, y_pred_svm, target_names=clases, output_dict=True)
reporte_cnn = classification_report(y_val, y_pred_cnn, target_names=clases, output_dict=True)

df_reporte_lr = pd.DataFrame(reporte_lr).transpose().round(4)
df_reporte_svm = pd.DataFrame(reporte_svm).transpose().round(4)
df_reporte_cnn = pd.DataFrame(reporte_cnn).transpose().round(4)

In [ ]:
df_reporte_lr

In [ ]:
df_reporte_svm

In [ ]:
df_reporte_cnn

### 8.1 Comparación por Clase

In [ ]:
metricas_por_clase = []

for clase in clases:
    metricas_por_clase.append({
        'Clase': clase,
        'LR Precision': reporte_lr[clase]['precision'],
        'LR Recall': reporte_lr[clase]['recall'],
        'LR F1': reporte_lr[clase]['f1-score'],
        'SVM Precision': reporte_svm[clase]['precision'],
        'SVM Recall': reporte_svm[clase]['recall'],
        'SVM F1': reporte_svm[clase]['f1-score'],
        'CNN Precision': reporte_cnn[clase]['precision'],
        'CNN Recall': reporte_cnn[clase]['recall'],
        'CNN F1': reporte_cnn[clase]['f1-score']
    })

df_metricas_clase = pd.DataFrame(metricas_por_clase).round(4)
df_metricas_clase

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

x = np.arange(len(clases))
ancho = 0.25

axes[0].bar(x - ancho, [reporte_lr[c]['precision'] for c in clases], ancho, 
           label='LR', alpha=0.8, color='#3498db', edgecolor='black')
axes[0].bar(x, [reporte_svm[c]['precision'] for c in clases], ancho, 
           label='SVM', alpha=0.8, color='#2ecc71', edgecolor='black')
axes[0].bar(x + ancho, [reporte_cnn[c]['precision'] for c in clases], ancho, 
           label='CNN', alpha=0.8, color='#e74c3c', edgecolor='black')
axes[0].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Precision', fontsize=12, fontweight='bold')
axes[0].set_title('Precision por Clase', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(clases)
axes[0].legend(fontsize=10)
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim([0, 1])

axes[1].bar(x - ancho, [reporte_lr[c]['recall'] for c in clases], ancho, 
           label='LR', alpha=0.8, color='#3498db', edgecolor='black')
axes[1].bar(x, [reporte_svm[c]['recall'] for c in clases], ancho, 
           label='SVM', alpha=0.8, color='#2ecc71', edgecolor='black')
axes[1].bar(x + ancho, [reporte_cnn[c]['recall'] for c in clases], ancho, 
           label='CNN', alpha=0.8, color='#e74c3c', edgecolor='black')
axes[1].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Recall', fontsize=12, fontweight='bold')
axes[1].set_title('Recall por Clase', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(clases)
axes[1].legend(fontsize=10)
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_ylim([0, 1])

axes[2].bar(x - ancho, [reporte_lr[c]['f1-score'] for c in clases], ancho, 
           label='LR', alpha=0.8, color='#3498db', edgecolor='black')
axes[2].bar(x, [reporte_svm[c]['f1-score'] for c in clases], ancho, 
           label='SVM', alpha=0.8, color='#2ecc71', edgecolor='black')
axes[2].bar(x + ancho, [reporte_cnn[c]['f1-score'] for c in clases], ancho, 
           label='CNN', alpha=0.8, color='#e74c3c', edgecolor='black')
axes[2].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[2].set_ylabel('F1-Score', fontsize=12, fontweight='bold')
axes[2].set_title('F1-Score por Clase', fontsize=14, fontweight='bold')
axes[2].set_xticks(x)
axes[2].set_xticklabels(clases)
axes[2].legend(fontsize=10)
axes[2].grid(axis='y', alpha=0.3)
axes[2].set_ylim([0, 1])

plt.tight_layout()
plt.savefig(ruta_figuras / '04_eval_05_metricas_por_clase.svg', format='svg', bbox_inches='tight')
plt.show()

## 9. Selección del Modelo Ganador

In [ ]:
modelo_ganador = df_ranking.iloc[0]['Modelo']
f1_macro_ganador = df_ranking.iloc[0]['F1 Macro Val']
f1_micro_ganador = df_ranking.iloc[0]['F1 Micro Val']
auc_pr_ganador = df_ranking.iloc[0]['AUC-PR Val']

df_ganador = pd.DataFrame({
    'Métrica': ['Modelo Seleccionado', 'F1 Macro Val', 'F1 Micro Val', 'AUC-PR Val', 'Ranking'],
    'Valor': [modelo_ganador, f1_macro_ganador, f1_micro_ganador, auc_pr_ganador, 1]
})

df_ganador

## 10. Conclusiones y Justificación Técnica

### 10.1 Análisis de Resultados

**Comparación de Modelos:**

1. **Logistic Regression (Modelo Base):**
   - Desempeño aceptable para un modelo simple
   - Entrenamiento rápido y eficiente
   - Limitado por la linealidad de las características PCA
   - Mejor para deployment en recursos limitados

2. **SVM (Modelo Avanzado 1):**
   - Mejora sobre Logistic Regression mediante kernels no lineales
   - Buena capacidad de generalización
   - Tiempo de entrenamiento moderado
   - Efectivo con características PCA de alta dimensión

3. **CNN (Modelo Avanzado 2):**
   - Aprende representaciones directamente de las imágenes
   - Mayor capacidad de capturar patrones complejos
   - Requiere más recursos computacionales
   - Beneficio de GPU/CUDA para entrenamiento

**Análisis de Convergencia (CNN):**
- Las curvas de pérdida muestran convergencia clara
- Pequeña diferencia entre train y validation indica buen ajuste
- No se observa overfitting significativo
- Early stopping implícito al guardar mejor modelo

**Métricas Clave:**
- **F1 Macro**: Métrica principal, importante para clases desbalanceadas
- **AUC-PR**: Confirma capacidad discriminativa en todas las clases
- **Matrices de Confusión**: Revelan patrones de confusión específicos por clase

### 10.2 Selección del Modelo Ganador

**Criterios de Selección:**
1. F1 Macro (peso 50%): Métrica balanceada entre precision y recall
2. F1 Micro (peso 20%): Desempeño global
3. AUC-PR (peso 20%): Capacidad discriminativa
4. Overfitting (peso 10%): Generalización

**Modelo Seleccionado:** El modelo con mejor ranking combinado

**Justificación Técnica:**
- Mejor F1 Macro indica mejor manejo del desbalance de clases
- AUC-PR alto confirma buena separación entre clases
- Bajo overfitting garantiza generalización a datos nuevos
- Matrices de confusión muestran menor número de clasificaciones erróneas

### 10.3 Recomendaciones para Implementación

**Para Producción:**
1. Si recursos son limitados: Usar Logistic Regression o SVM
2. Si se dispone de GPU: CNN ofrece mejor desempeño
3. Implementar ensemble de los 3 modelos para mayor robustez

**Monitoreo:**
- Tracking de F1 Macro en producción
- Detección de drift en distribución de clases
- Reentrenamiento periódico con datos nuevos

### 10.4 Limitaciones del Dataset

1. **Tamaño:** Dataset relativamente pequeño (2,727 imágenes totales)
2. **Desbalance:** Clase 'plastic' significativamente minoritaria
3. **Variabilidad:** Limitada en condiciones de iluminación y fondos
4. **Generalización:** Posible overfitting a condiciones específicas del dataset

### 10.5 Trabajo Futuro

**Mejoras Propuestas:**
1. Aumentar dataset con más imágenes reales
2. Transfer learning con modelos pre-entrenados (ResNet, EfficientNet)
3. Técnicas de balanceo más sofisticadas (SMOTE para imágenes)
4. Explorar arquitecturas más profundas (ResNet, DenseNet)
5. Implementar data augmentation más agresivo
6. Probar ensemble methods (voting, stacking)
7. Optimización de hiperparámetros con Bayesian Optimization más extensiva

**Validación Adicional:**
- Test set independiente para validación final
- Cross-validation estratificado más exhaustivo
- Evaluación en condiciones reales de operación